In [103]:
import dimod
from data import relations, starting_point, labels

In [110]:
model = dimod.BQM("BINARY")

P = 20 # the lagrange term for penality
valid_relation = lambda relation: relation[0] != relation[1] and relation[1] != starting_point  # only nodes that goes forward and doesn't create any loops

current_node = starting_point 
all_nodes_terms_contraint = []
visited_nodes_per_step_constraint_terms = []
visited_target_nodes_constraint_terms_per_label = {label:[] for label in labels} 

for relation, distance in relations.items():
    if(not valid_relation(relation)):
        continue

    if(relation[0] != current_node):
        # before start adding the relations from a different node
        # add a constraint to ensure that only one node (-1 as C) is visited at a time
        model.add_linear_equality_constraint(visited_nodes_per_step_constraint_terms, P, -1) # 
        visited_nodes_per_step_constraint_terms = []
        current_node = relation[0]

    visited_nodes_per_step_constraint_terms.append((relation,1))
    all_nodes_terms_contraint.append((relation,1))

    # keep track of target nodes that we visited, just one must be visited in the whole path
    visited_target_nodes_constraint_terms_per_label[relation[1]].append((relation,1))
    
    model.add_linear(relation,distance)

    
# ensure that our resulting combination contains only 4 ones (-4 as C)
model.add_linear_equality_constraint(all_nodes_terms_contraint, P, -4)

# apply the visited target constraints, as I told before, only one must be visited
for targets in visited_target_nodes_constraint_terms_per_label.values():
    model.add_linear_equality_constraint(targets, P, -1)

In [132]:
sampler = dimod.ExactSolver()
results = sampler.sample(model)
shortest_path = results.lowest()

In [144]:
df = results.to_pandas_dataframe(sample_column=True) 
df.to_csv("qubo-results.csv", sep=',', encoding='utf-8', index=False, header=True)

In [143]:
shortest_path_data = next(shortest_path.data()).sample
path_size = 0

print("Path: ", end="")

for label, value in shortest_path_data.items():
    if(value == 0):
        continue
    path_size += relations[label]        
    print(label, end=" ")

print(f"\nPath size: {path_size}")

Path: AB BC CD DE 
Path size: 13.1
